# Generate SMA using dedicated service - C#

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *simple moving average - SMA* from a dedicated data service.

### Inputs/outputs
Simple moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

Basically, SMA is formed by computing the average price of a security over a specific number of points. As its name implies, simple moving average is an average that moves. Old data is dropped and new data becomes available, causing the average to move along the time.

N-points SMA data point is computed by adding up the last N period's trade prices and then dividing by N as follows:

$$ SMA = \frac{\Sigma Price n}{N} $$
Where:
* *Price n* is the trade price of the *period n*
* *N* is the count of time periods

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve SMA indicator from the hosted service. The queried endpoint in this script are:
* *TickSmaService*: to directly retrieve SMA data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run SMA sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Core;
using Grpc.Net.Client;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *SMA* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Simple moving average request parameters

#### 3.1 Instrument selection

In [4]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 SMA parameters

In [5]:
var field = SmaPrice.Trade;
var longSma = 200;
var shortSma = 50;
var sampling = new Duration { Seconds = 1 * 60 };

#### 3.3 Time period delimitation

In [6]:
var today = DateTime.Today;
var dateIntervals = new DateInterval
{
    StartDate = new Date { Year = 2022, Month = 02, Day = 18 },
    EndDate = new Date { Year = 2022, Month = 02, Day = 18 }
};

var timeInterval = new TimeInterval
{
    StartTime = new TimeOfDay { Hours = 08, Minutes = 00, Seconds = 00 },
    EndTime = new TimeOfDay { Hours = 20, Minutes = 00, Seconds = 00 }
};

#### 3.4 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [7]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints();
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);   

The following code snippets create requests for: *Long SMA*, *Short SMA* and *Prices*:

In [8]:
// Generate 200-points SMA request
var smaLongRequest = new TickSmaRequest()
{
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Constraints = constraints,  
    Field = field,
    Length = longSma, 
    Sampling = sampling
};

// Generate 50-points SMA request
var smaShortRequest = new TickSmaRequest()
{
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Constraints = constraints,  
    Field = field,
    Length = shortSma, 
    Sampling = sampling
};

// Instantiate the tick SMA service
var service = new TickSmaService.TickSmaServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve long SMA data

In [9]:
// smalong includes long SMA data points
var smalong = new List<TickSmaResponse>();

var call = service.TickSma(smaLongRequest, headers);
await foreach (var data in call.ResponseStream.ReadAllAsync())
{
    smalong.Add(data);
}
//display(smalong);

#### 4.2 Retrieve short SMA data

In [10]:
// smashort includes short SMA data points
var smashort = new List<TickSmaResponse>();

var call = service.TickSma(smaShortRequest, headers);
await foreach (var data in call.ResponseStream.ReadAllAsync())
{
    smashort.Add(data);
}
//display(smashort);

#### 4.4 Plot SMAs and prices
Plot SMA and prices request' results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. Plot a simple line graph with the SMA data points and prices: 

In [11]:
// Plot simultaneously prices and simple moving averages
var smaPlots = new List<Scattergl>
{
    new Scattergl
    {
        name = "200-points SMA",
        mode = "lines",
        x = smalong.Select(l => l.TimeStamp.ToDateTime()),
        y = smalong.Select(l => l.Average)
    },
    new Scattergl
    {
        name = "50-points SMA",
        mode = "lines",
        x = smashort.Select(l => l.TimeStamp.ToDateTime()),
        y = smashort.Select(l => l.Average)
    },
    new Scattergl
    {
        name = "Prices",
        mode = "lines",
        x = smashort.Select(l => l.TimeStamp.ToDateTime()),
        y = smashort.Select(l => l.Value)
    }
};
var chart = Chart.Plot(smaPlots);
chart.WithTitle($"SMAs and prices for {ticker}");
chart.WithXTitle($"Time");
chart.WithYTitle($"Price");
chart.Width = 1500;
chart.Height = 800;
display(chart);

<!DOCTYPE html>